# TP Explicabilité - SHAP (SHapley Additive exPlanations)

**Auteur:** Sandie Cabon  
**Date:** 2 février 2026

Ce notebook permet d'utiliser SHAP pour expliquer les prédictions du modèle.

## Import des bibliothèques

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from helping_functions_et import infer_column_types
import joblib
import shap

## Configuration (NE PAS MODIFIER)

In [ ]:
print("############### CHARGEMENT DES DONNÉES #####################")

# close older figures
plt.close("all")

# load heart failure dataset
dataset = pd.read_csv("heart_failure_dataset_test.csv")

# apply good type to dataframe (custom function)
dataset = infer_column_types(dataset)

# separate feat and target values
dataset_feat = dataset.drop("DEATH_EVENT", axis=1)
dataset_target = dataset["DEATH_EVENT"]
feat_names = list(dataset_feat.columns)

# load the pipeline (composed by a preprocessor and a model)
loaded_RF = joblib.load('death_RF_predictor.pkl')

preprocessor = loaded_RF.named_steps['preprocessor']
model = loaded_RF.named_steps['model']

# settings for reproducibility
random_state = 12
np.random.seed(12)

## V. SHAP (SHapley Additive exPlanations)

### Préparation des données et calcul des SHAP values

In [ ]:
print("############ V. SHAP (SHapley Additive exPlanations) ############")

# Les fonctions de Shap ne prennent pas en charge le preprocessing (mise à l'echelle). 
# Il faut donc le faire en amont
# Effectuer le prétraitement manuellement sur les données de test
# aide : utiliser la fonction transfrom du preprocessor sur le dataFrame contenant les variables
feat_scaled = [] # à modifier
dataset_scaled = pd.DataFrame(feat_scaled, columns=feat_names)
dataset_scaled = infer_column_types(dataset_scaled)

# Créer un objet explainer SHAP en utilidans 
explainer = shap.Explainer(model)
# Calculer les Shapley values pour l'ensemble de test
# shap_values = explainer.shap_values(dataset_scaled) # à décommenter quand le dataset_scaled est prêt

### Choix de la méthode

Nommer la methode pour executer la bonne partie du code : "summary", "beeswarm", "waterfall" or "all"

In [ ]:
# Nommer la methode pour executer la bonne partie du code : "summary", "beeswarm", "waterfall" or "all"
method = "summary"

### Summary plot
compléter la fonction shap.summary_plot() https://shap-lrjball.readthedocs.io/en/latest/generated/shap.summary_plot.html

In [ ]:
if method == "summary" or method == "all":
    # generation du graphe
    # compléter la fonction shap.summary_plot() https://shap-lrjball.readthedocs.io/en/latest/generated/shap.summary_plot.html
    plt.figure()
    #shap.summary_plot(..., ..., plot_type="bar") 
    plt.tight_layout()
    plt.title('Summary')
    plt.show()

### Beeswarm plot

In [ ]:
if method == "beeswarm" or method == "all":
    shap_values = explainer(dataset_scaled)
    
    # generation du graphe
    shap.plots.beeswarm(shap_values[:,:,1], max_display=15)
    plt.tight_layout()
    plt.title('Beeswarm')
    plt.show()

### Waterfall plot pour une observation spécifique

In [ ]:
if method == "waterfall" or method == "all":
    # choix de l'observation à étudier. Renseigner l'index.
    obs_index = ...
    print("Le label associé à cette observation est : %s " %dataset_target[obs_index])
    
    # generation du graphe
    # compléter la fonction shap.plots.waterfall
    plt.figure()
    shap.plots.waterfall(shap_values=shap_values[..., :,1],
                         max_display=20,show=False)
    
    plt.title(f'SHAP effects Observation {obs_index}')
    plt.tight_layout()
    plt.show()

## Notes et Aides

### AIDE 1
Les fonctions de Shap ne prennent pas en charge le preprocessing (mise à l'echelle). 
Dans les affichages, nous observons donc les valeurs mises à l'echelle. 
Cela rends plus difficile l'interprétation.

Vous pouvez utiliser la fonction `get_original_value(idx, feat_name, original_data)`
qui vous permettra de convertir une valeur mise à l'echelle en sa valeur originale.
Importer là au préalable. Vous pourrez ensuite l'executer dans la console.

### AIDE 2
Pour selectionner des erreurs de type Faux Négatifs, Faux Positifs:
- Utiliser la méthode `predict()` de model sur les données mises à l'échelle
- Ou utiliser la méthode `predict()` du pipeline (loaded_RF) sur les données originales
- Comparer avec le contenu de la variable `dataset_target` pour trouver des observations d'intérêt